In [1]:
import torch
import Models.misc as misc
import Inference.Kolmogorov.performance as performance
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Emu_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_best.p'
CT_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250715_210115-naiteaau/files/checkpoint_last.p'

CT = misc.load_diffusion_model(CT_file_path).to(device)
Emu = misc.load_model(Emu_file_path).to(device)

In [2]:
data_dict = torch.load("/scratch/ql2221/thermalizer_data/kolmogorov/reynold10k/test_long.p")

In [3]:
data = data_dict["data"]
print(data.shape)
x = data[0,:,:,:]
x = x.unsqueeze(0).to(device)
print(x.shape)
x = x/4.44

torch.Size([5, 70000, 64, 64])
torch.Size([1, 70000, 64, 64])


In [4]:
delta = torch.tensor([20]).to(device)
rollout, _ = performance.run_conditional_emu(x[:,0:1], Emu, CT, n_steps=20000, delta = delta, denoising_steps=5, freq = 25, silent=True, sigma=None, Regression = True)

Denoising: 100%|██████████| 3/3 [00:00<00:00, 110.98it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 112.49it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 111.53it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 110.94it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 3/3 [00:00<00:00, 111.19it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 110.98it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 111.53it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 5/5 [00:00<00:00, 111.45it/s]
Denoising: 0it [00:00, ?it/s]
Denoising: 100%|██████████| 3/3 [00:00<00:00, 109.67it/s

In [5]:
import matplotlib.pyplot as plt
from matplotlib import animation
def make_movie(state_vector, save_path="emulator_movie.mp4", fps=30, vmin=None, vmax=None, stride=10):
    """
    Create a movie from a 4D tensor of shape (batch, time, height, width), using only every `stride`-th frame.
    
    Parameters:
    - state_vector: torch.Tensor of shape (B, T, H, W)
    - save_path: output video file path
    - fps: frames per second for the movie
    - vmin, vmax: color scale
    - stride: only include every `stride`th frame in the video
    """
    B, T, H, W = state_vector.shape
    data = state_vector[0].cpu().numpy()

    fig, ax = plt.subplots()
    im = ax.imshow(data[0], cmap="viridis", vmin=vmin, vmax=vmax)
    ax.set_title("Timestep 0")
    fig.colorbar(im, ax=ax)

    # Frame indices to use
    frame_indices = list(range(0, T, stride))

    def update(frame_idx):
        frame = frame_indices[frame_idx]
        im.set_data(data[frame])
        ax.set_title(f"Timestep {frame}")
        return im,

    ani = animation.FuncAnimation(fig, update, frames=len(frame_indices), blit=False, repeat=False)
    ani.save(save_path, fps=fps)
    plt.close(fig)
    print("movie saved")

In [6]:
make_movie(rollout, save_path="rollout.mp4", fps=30)

movie saved
